In [11]:
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

In [12]:
# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

In [28]:
# データ読み込み
x = [1,2,3]
for i in x:
    df_S{}.format(i) = pd.read_csv("../data/Summer{i}_0412.csv".format(i),encoding="shift-jis")

SyntaxError: invalid syntax (Temp/ipykernel_13096/380095167.py, line 4)

In [19]:
# データ読み込み
df_S1 = pd.read_csv("../data/Summer1_0412.csv",encoding="shift-jis")#pandasというライブラリのread_csv()関数を使用するという意味
#変数のあとに.（ピリオド）を打って、tabキーを押すと変数（オブジェクト）が持つ機能（プロパティ・メソッド）の一覧が表示される。
#相対パスで指定しているので、../でひとつ上のディレクトリに戻り、そこから下のディレクトリを指定する。

In [20]:
# データ読み込み
df_S2 = pd.read_csv("../data/Summer2_0412.csv",encoding="shift-jis")#pandasというライブラリのread_csv()関数を使用するという意味
#変数のあとに.（ピリオド）を打って、tabキーを押すと変数（オブジェクト）が持つ機能（プロパティ・メソッド）の一覧が表示される。
#相対パスで指定しているので、../でひとつ上のディレクトリに戻り、そこから下のディレクトリを指定する。

In [21]:
# データ読み込み
df_S3 = pd.read_csv("../data/Summer3_0412.csv",encoding="shift-jis")#pandasというライブラリのread_csv()関数を使用するという意味
#変数のあとに.（ピリオド）を打って、tabキーを押すと変数（オブジェクト）が持つ機能（プロパティ・メソッド）の一覧が表示される。
#相対パスで指定しているので、../でひとつ上のディレクトリに戻り、そこから下のディレクトリを指定する。

In [14]:
df_S1.head(3)#dataframeの先頭3行を確認

,製品コード,科目コード,採点者コード,合計 / 合計 / 採点所要時間,合計 / 合計 / 採点済枚数,合計 / 合計 / バツウチ済枚数,実枚数,ペース
0,111103.0,10.0,3.050016e+09,122506.0,8057.0,942.0,7115.0,209.1
1,111103.0,10.0,3.050727e+09,123770.0,8585.0,967.0,7618.0,221.6
2,111103.0,30.0,3.050742e+09,155672.0,6014.0,1622.0,4392.0,101.6


In [24]:
# 重複確認
df_S1 = pandas_tool.check_dup(df,["科目コード","採点者コード"],True)

  var_name  dup_cnt
0    科目コード      958
1   採点者コード        0


In [25]:
# 重複確認
df_S2 = pandas_tool.check_dup(df,["科目コード","採点者コード"],True)

  var_name  dup_cnt
0    科目コード      958
1   採点者コード        0


In [26]:
# 重複確認
df_S3 = pandas_tool.check_dup(df,["科目コード","採点者コード"],True)

  var_name  dup_cnt
0    科目コード      958
1   採点者コード        0


In [15]:
# 型チェック
df_ret = pandas_tool.type(df)

In [16]:
df_ret

製品コード                float64
科目コード                float64
採点者コード               float64
合計 / 合計 / 採点所要時間     float64
合計 / 合計 / 採点済枚数      float64
合計 / 合計 / バツウチ済枚数    float64
実枚数                  float64
ペース                  float64
dtype: object

In [17]:
# 基礎集計
df_ret = pandas_tool.summary(df,True)

・統計量
          製品コード       科目コード        採点者コード  合計 / 合計 / 採点所要時間  合計 / 合計 / 採点済枚数  \
count     961.0  961.000000  9.610000e+02      9.610000e+02       961.000000   
mean   111103.0   19.635796  3.068431e+09      1.482047e+05      8085.276795   
std         0.0    8.289836  3.693403e+06      1.394240e+05     10359.574469   
min    111103.0   10.000000  3.050016e+09      6.900000e+01         1.000000   
25%    111103.0   10.000000  3.070021e+09      6.453000e+04      3832.000000   
50%    111103.0   20.000000  3.070054e+09      1.077960e+05      5218.000000   
75%    111103.0   30.000000  3.070064e+09      1.779920e+05      8997.000000   
max    111103.0   30.000000  3.070073e+09      1.713518e+06    136214.000000   

       合計 / 合計 / バツウチ済枚数           実枚数          ペース  
count         961.000000    961.000000   961.000000  
mean         2714.369407   5370.907388   209.851717  
std          8510.868588   3913.837796   227.539917  
min             0.000000      1.000000     4.900000  
25% 

In [14]:
# 欠損確認
df_ret = pandas_tool.check_missing(df,True)

      製品コード  科目\nコード  採点者コード  パートNo  合計 / 採点所要時間  合計 / 採点済枚数  合計 / バツウチ済枚数  \
True    961      961     961    961          961         961           961   

      実枚数  ペース  
True  961  961  


In [19]:
# カテゴリ集計
df_ret = pandas_tool.freq(df,["科目\nコード","パートNo"],True)

101    348
301    313
203    300
Name: 科目\nコード, dtype: int64
3     208
1     191
2     187
5     129
6     116
4      63
7      47
9       8
8       7
10      5
Name: パートNo, dtype: int64


In [22]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df_ret = pandas_tool.tabulate(df=df,col=['科目コード'],row=['パートNo'],var=['ペース'],func=np.mean)

KeyError: 'パートNo'

In [22]:
df_ret

ペース                        
科目
コード         101         203         301
パートNo                                     
1       349.610909  117.378505  744.862069
2       167.630508  149.574528  624.654545
3       136.038776  151.259770  233.665217
4       337.542105    0.000000  558.580000
5       181.893333    0.000000  116.017857
6       364.448148    0.000000   90.282022
7       317.716667    0.000000   42.909756
8       327.400000    0.000000    0.000000
9       566.425000    0.000000    0.000000
10      767.880000    0.000000    0.000000

In [33]:
# Merge
ans_df = pd.read_csv("gender_submission.csv")
df_ret = pandas_tool.merge(df,ans_df,"PassengerId","left",view=True)

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:891
df2のオブザベーション:418
mergeのオブザベーション:891


In [34]:
# Rank
df_ret,ret = pandas_tool.rank(df,var='Fare',num=10,check=True)

In [87]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_time = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['合計 / 合計 / 採点所要時間'],func=np.sum)

In [108]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_pace_mean = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['ペース'],func=np.mean)c

In [91]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_pace_min = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['ペース'],func=np.min)

In [92]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_pace_max = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['ペース'],func=np.max)

In [93]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_pace_median = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['ペース'],func=np.median)

In [99]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_pace_var = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['ペース'],func=np.var)

In [104]:
# 複雑な集計
#tabulate(df,列、行、目的変数、集計方法)
df1_ret_pace_size = pandas_tool.tabulate(df=df1,col=['科目コード'],row=['割当'],var=['ペース'],func=np.size)

In [35]:
# Apply Rank　エラー出るから放置
del df["Fare_rank"]
pandas_tool.apply_rank(df,ret)

AttributeError: 'DataFrame' object has no attribute 'ix'